In [1]:
from dataloader_video_flow import *

### Diff Dataset loader

In [2]:
near_ood = True
dataset_lst = ['HMDB', 'EPIC', 'UCF','Kinetics']
dataset = dataset_lst[3]

In [ ]:
from mmaction.apis import init_recognizer
import torch
import torch.nn as nn

config_file = 'configs/recognition/slowfast/slowfast_r101_8x8x1_256e_kinetics400_rgb.py'
config_file_flow = 'configs/recognition/slowonly/slowonly_r50_8x8x1_256e_kinetics400_flow.py'

device = 'cuda:0' # or 'cpu'
device = torch.device(device)

v_dim = 2304
f_dim = 2048


if near_ood:
    if dataset == 'HMDB':
        num_class = 25
    elif dataset == 'UCF':
        num_class = 50
    elif dataset == 'Kinetics':
        num_class = 129
else:
    if dataset == 'HMDB':
        num_class = 43
    elif dataset == 'Kinetics':
        num_class = 229
# num_class = 25

print("####",num_class)
# build the model from a config file and a checkpoint file
model = init_recognizer(config_file, device=device, use_frames=True)
model.cls_head.fc_cls = nn.Linear(v_dim, num_class).cuda()
cfg = model.cfg
model = torch.nn.DataParallel(model)

model_flow = init_recognizer(config_file_flow, device=device,use_frames=True)
model_flow.cls_head.fc_cls = nn.Linear(f_dim, num_class).cuda()
cfg_flow = model_flow.cfg

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dataset = EPICDOMAIN(split='train', eval=True, cfg=cfg, cfg_flow=cfg_flow, datapath='/data/shared_dataset/data/Kinetics-600/', dataset=dataset, near_ood=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=4, shuffle=False,
                                                pin_memory=(device.type == "cuda"), drop_last=False)

In [ ]:
for clip, spectrogram, labels in train_dataloader:
    print(clip['imgs'].numpy().shape)
    print(spectrogram['imgs'].numpy().shape)
    break

In [5]:
clip_lst1 = []
spectrogram_lst1 = []
for clip, spectrogram, labels in train_dataloader:
    clip_lst1.append(clip['imgs'].numpy())
    spectrogram_lst1.append(spectrogram['imgs'].numpy())

directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241
directory:  
frame_num:  241


KeyboardInterrupt: 

In [ ]:
# clip_lst1[0].shape20, clip_lst1[2].shape, spectrogram_lst1[0].shape, spectrogram_lst1[1].shape

In [ ]:
# shape_lst = []
# shape_lst1 = []
# for i in range(len(clip_lst1)):
#     shape_lst.append(clip_lst1[i].shape)
#     shape_lst1.append(spectrogram_lst1[i].shape)
# print(set(shape_lst))
# print(set(shape_lst1))
    # print(clip_lst1[i].shape, spectrogram_lst1[i].shape)

In [ ]:
if near_ood:
    ood_str = 'near'
else:
    ood_str = 'far'


# Save with automatic keys
np.savez(f'clip_lst_{dataset}_{ood_str}.npz', *clip_lst1)
np.savez(f'spectrogram_lst_{dataset}_{ood_str}.npz', *spectrogram_lst1)


# clip_lst = np.stack(clip_lst1)
# spectrogram_lst = np.stack(spectrogram_lst1)
# np.save(f'clip_lst_{dataset}_{ood_str}.npy', clip_lst)
# np.save(f'spectrogram_lst_{dataset}_{ood_str}.npy', spectrogram_lst)
print('file saved.')

In [ ]:
# clip_lst.shape

In [ ]:
# spectrogram_lst.shape